# Importation des données

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from collections import OrderedDict
from pprint import pprint

In [3]:
data_beat_aml = pd.read_csv("BEATAMLdata/BEATAML_Cliniques.csv", comment="#")
data_NPM1_mut = pd.read_csv("MUTdata/NPM1_alt.tsv", sep="\t", comment="#")
data_DNMT3A_mut = pd.read_csv("MUTdata/DNMT3A_alt.tsv", sep="\t", comment="#")

# NPM1

## Avoir les types de mutations d'un gène 

Pour caractériser une mutation unique : 
* Sa position (feature "localisation")
* Sa nature (feature "type_alt")
* Séquences référente et altérée (features "seq_ref" et "seq_alt")

In [4]:
def get_types_mutations(datamut, to_csv=False):
    MutationsCaracteristics = datamut[["localisation", "type_alt", "seq_ref", "seq_alt"]]
    MutationsCaracteristics = MutationsCaracteristics.drop_duplicates()
    MutationsCaracteristics.insert(0, "mutation_name", [f"mut{i}" for i in range(1, len(MutationsCaracteristics)+1)])

    if to_csv:
        MutationsCaracteristics.to_csv("TypesMutations.csv", index=False)
        
    return MutationsCaracteristics

MutationsCaracteristicsNPM1 = get_types_mutations(data_NPM1_mut)

## Relier les patients au type de mutation qu'ils ont

In [5]:
def link_patients_and_mutations(MutationsCaracteristics, datamut):
    MutationAndPatients_dico = {mut:[] for mut in MutationsCaracteristics["mutation_name"]}
    # print(MutationAndPatients_dico)

    for _, patient in datamut.iterrows():
        patient_loc = patient["localisation"]
        patient_type = patient["type_alt"]
        patient_ref = patient["seq_ref"]
        patient_alt = patient["seq_alt"]
        for _, mut in MutationsCaracteristics.iterrows():
            if mut["localisation"] == patient_loc and mut["type_alt"] == patient_type and mut["seq_ref"] == patient_ref and mut["seq_alt"] == patient_alt:
                MutationAndPatients_dico[mut["mutation_name"]].append(patient["ID_sample"][:6])
                #on enlève tous les caractères au delà de 6 pour ne garder que l'ID sample
    return MutationAndPatients_dico


MutationAndPatients_dicoNPM1 = link_patients_and_mutations(MutationsCaracteristicsNPM1, data_NPM1_mut)

In [6]:
MutationAndPatients_dicoNPM1 = {k: v for k, v in MutationAndPatients_dicoNPM1.items() if len(v) >= 3}

pprint(OrderedDict(MutationAndPatients_dicoNPM1))

OrderedDict([('mut1',
              ['BA2160',
               'BA2303',
               'BA2590',
               'BA2924',
               'BA3083',
               'BA3089',
               'BA3099',
               'BA3119',
               'BA3135',
               'BA3157',
               'BA3164',
               'BA3176',
               'BA3183',
               'BA3192',
               'BA3197',
               'BA3200',
               'BA3216',
               'BA3225',
               'BA3232',
               'BA3234',
               'BA3235',
               'BA3258',
               'BA3266',
               'BA3269',
               'BA3277',
               'BA3293',
               'BA3297',
               'BA3299',
               'BA3306',
               'BA3310',
               'BA3328',
               'BA3332',
               'BA3333',
               'BA3347',
               'BA3352',
               'BA3375',
               'BA3385',
               'BA3390',
               'BA3400',
   

Tous les individus qui ont une mutation sur le gène NPM1 :

In [7]:
def get_patients_with_gene_mut(datamut):
    ind_mut = list(datamut["ID_sample"])
    ind_mut = [ind[:6] for ind in ind_mut]
    return ind_mut

ind_NPM1mut = get_patients_with_gene_mut(data_NPM1_mut)
print(ind_NPM1mut)

['BA2160', 'BA2303', 'BA2590', 'BA2924', 'BA3083', 'BA3089', 'BA3099', 'BA3119', 'BA3135', 'BA3157', 'BA3164', 'BA3176', 'BA3183', 'BA3192', 'BA3197', 'BA3200', 'BA3216', 'BA3225', 'BA3232', 'BA3234', 'BA3235', 'BA3258', 'BA3266', 'BA3269', 'BA3277', 'BA3293', 'BA3297', 'BA3299', 'BA3306', 'BA3310', 'BA3328', 'BA3332', 'BA3333', 'BA3347', 'BA3352', 'BA3375', 'BA3385', 'BA3390', 'BA3400', 'BA3421', 'BA3422', 'BA3423', 'BA3431', 'BA3433', 'BA3446', 'BA3451', 'BA3453', 'BA3454', 'BA2003', 'BA2020', 'BA2032', 'BA2054', 'BA2065', 'BA2098', 'BA2104', 'BA2106', 'BA2110', 'BA2123', 'BA2124', 'BA2133', 'BA2139', 'BA2171', 'BA2200', 'BA2201', 'BA2211', 'BA2226', 'BA2233', 'BA2236', 'BA2239', 'BA2240', 'BA2242', 'BA2244', 'BA2249', 'BA2277', 'BA2280', 'BA2301', 'BA2376', 'BA2386', 'BA2410', 'BA2418', 'BA2450', 'BA2452', 'BA2473', 'BA2474', 'BA2477', 'BA2479', 'BA2480', 'BA2485', 'BA2512', 'BA2523', 'BA2552', 'BA2563', 'BA2579', 'BA2588', 'BA2662', 'BA2704', 'BA2705', 'BA2726', 'BA2744', 'BA2746',

## Avoir tous les individus présentés par leur Sample ID

In [8]:
samples = data_beat_aml[["dbgap_dnaseq_sample", "dbgap_rnaseq_sample"]]
ind_beatAML = []

for _, sample in samples.iterrows():
    if pd.isna(sample["dbgap_dnaseq_sample"]): #and pd.isna(sample["dbgap_rnaseq_sample"]):
        sample_id = sample["dbgap_rnaseq_sample"][:6]
    else:
        sample_id = sample["dbgap_dnaseq_sample"][:6]
    ind_beatAML.append(sample_id)

print(ind_beatAML)
print(len(ind_beatAML))

['BA2102', 'BA2005', 'BA3074', 'BA2555', 'BA3073', 'BA2452', 'BA2673', 'BA2803', 'BA2922', 'BA2216', 'BA2859', 'BA2336', 'BA2708', 'BA2643', 'BA2265', 'BA2460', 'BA2067', 'BA2881', 'BA2012', 'BA2352', 'BA2630', 'BA2335', 'BA2707', 'BA2825', 'BA2435', 'BA2916', 'BA3035', 'BA3039', 'BA3172', 'BA2409', 'BA3021', 'BA2480', 'BA2496', 'BA2136', 'BA3037', 'BA2831', 'BA2847', 'BA2045', 'BA2236', 'BA3439', 'BA2918', 'BA2172', 'BA3042', 'BA2865', 'BA2923', 'BA2003', 'BA2751', 'BA2055', 'BA3395', 'BA2286', 'BA2763', 'BA2488', 'BA3379', 'BA2887', 'BA2712', 'BA2858', 'BA3354', 'BA3409', 'BA3097', 'BA3288', 'BA3387', 'BA3159', 'BA2068', 'BA2104', 'BA3124', 'BA3106', 'BA3259', 'BA2921', 'BA2449', 'BA2360', 'BA2802', 'BA2235', 'BA3038', 'BA3006', 'BA2245', 'BA2667', 'BA2239', 'BA2359', 'BA2782', 'BA2649', 'BA2117', 'BA2257', 'BA2258', 'BA2592', 'BA2861', 'BA2684', 'BA2598', 'BA2134', 'BA2337', 'BA2909', 'BA2429', 'BA2653', 'BA2947', 'BA2032', 'BA2162', 'BA2991', 'BA2564', 'BA3043', 'BA2597', 'BA2315',

## Tests : table de contingence (voir une corrélation entre une mutation et une feature)

Test avec les mutations de NPM1 et la feature ELN2017.

In [9]:
filtered_data_beat_aml = data_beat_aml[["dbgap_dnaseq_sample", "dbgap_rnaseq_sample", "ELN2017"]]

In [10]:
def create_contingency_table(MutationAndPatients_dico, ind_mut):
    Table = pd.DataFrame(index=MutationAndPatients_dico.keys(), columns=list(np.unique(filtered_data_beat_aml["ELN2017"])))
    Table = Table.fillna(0)
    for i, ind in enumerate(ind_beatAML):
        if ind in ind_mut:
            value = filtered_data_beat_aml.loc[i, "ELN2017"]
            mutType = [mut for mut, patients in MutationAndPatients_dico.items() if ind in patients] 
            if len(mutType) == 1: #on vérifie que la liste n'est pas vide (que le patient a bien une mutation)
                mutType = mutType[0] #pour plus tard : gérer plusieurs mutations (s'il y en a)
                Table.loc[mutType, value] += 1
    
    pseudo_count = 0.5 #éviter les fréquences à 0
    # Table += pseudo_count

    return Table


TableNPM1 = create_contingency_table(MutationAndPatients_dicoNPM1, ind_NPM1mut)
TableNPM1 = TableNPM1[["Favorable", "FavorableOrIntermediate", "Intermediate", "IntermediateOrAdverse", "Adverse"]]
# TableNPM1 = TableNPM1[["Favorable", "Intermediate", "Adverse"]]
TableNPM1

/tmp/ipykernel_6412/1615703540.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Table = Table.fillna(0)


,Favorable,FavorableOrIntermediate,Intermediate,IntermediateOrAdverse,Adverse
mut1,64,7,19,0,2
mut2,2,0,2,0,0
mut4,8,0,2,0,0


In [11]:
# Effectuer le test du chi-carré d'indépendance sur la table de contingence globale
chi2, p, dof, expected = chi2_contingency(TableNPM1)

# Afficher les résultats du test
print("\nRésultats du test du chi-carré d'indépendance:")
print(f"Chi2: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:")
# print(expected)

# Interprétation des résultats
alpha = 0.05
if p < alpha:
    print("\nIl existe une association significative entre les types de mutations et la variable ELN2017 (rejeter H0).")
else:
    print("\nIl n'existe pas d'association significative entre les types de mutations et la variable ELN2017 (ne pas rejeter H0).")

ValueError: The internally computed table of expected frequencies has a zero element at (0, 3).

In [ ]:
# Initialiser un dictionnaire pour stocker les résultats du test exact de Fisher
results = {}

# Effectuer le test exact de Fisher pour chaque mutation
for mutation in TableNPM1.index:
    # Créer une table de contingence pour la mutation actuelle (1 vs all)
    mutation_counts = TableNPM1.loc[mutation]
    other_counts = TableNPM1.drop(mutation).sum()
    
    mutation_table = pd.DataFrame({
        'Feature': TableNPM1.columns,
        'Mutation': mutation_counts,
        'Others': other_counts
    }).set_index('Feature').T

    # Normalisation (évite l'effet du nombre inégal de patients par mutation)
    # mutation_table = mutation_table.div(mutation_table.sum(axis=1), axis=0)
    # print(mutation_table)
    
    # Test exact de Fisher pour chaque catégorie de la feature
    fisher_results = {}
    for category in mutation_table.columns:
        # Construire la table 2x2 pour le test de Fisher
        table_2x2 = np.array([
            [mutation_table.loc["Mutation", category], mutation_table.loc["Mutation"].sum() - mutation_table.loc["Mutation", category]],
            [mutation_table.loc["Others", category], mutation_table.loc["Others"].sum() - mutation_table.loc["Others", category]]
        ])

        # Vérifier que la table contient des valeurs non nulles
        if np.all(table_2x2 > 0):
            odds_ratio, p_fisher = fisher_exact(table_2x2, alternative="two-sided")
            fisher_results[category] = {"Odds Ratio": odds_ratio, "p-value (Fisher)": p_fisher}
        else:
            fisher_results[category] = {"Odds Ratio": None, "p-value (Fisher)": None}

    # Ajouter les résultats de Fisher au dictionnaire principal
    results[mutation] = fisher_results

# Affichage des résultats
# for mutation, fisher_result in results.items():
#     print(f"Mutation: {mutation}")
#     for category, result in fisher_result.items():
#         print(f"  {category}: OR={result['Odds Ratio']}, p-value={result['p-value (Fisher)']}")
#     print("\n")

# Interprétation des résultats (seuil alpha = 0.05)
alpha = 0.05
for mutation, fisher_result in results.items():
    for category, result in fisher_result.items():
        if result['p-value (Fisher)'] is not None and result['p-value (Fisher)'] < alpha:
            print(f"La catégorie {category} est significativement associée à la mutation {mutation} selon le test exact de Fisher.")

In [ ]:
from statsmodels.stats.contingency_tables import Table

contingency_table = Table(TableNPM1)

res = contingency_table.test_nominal_association()

print("\nRésultats du test:")
print(res)

alpha = 0.05
if res.pvalue < alpha:
    print("\nIl existe une association significative entre les types de mutations et la variable ELN2017 (rejeter H0).")
else:
    print("\nIl n'existe pas d'association significative entre les types de mutations et la variable ELN2017 (ne pas rejeter H0).")


Résultats du test:
df          8
pvalue      0.6094658899292145
statistic   6.337652886359652

Il n'existe pas d'association significative entre les types de mutations et la variable ELN2017 (ne pas rejeter H0).


Odds ratio (pour un tableau où les colonnes/lignes : Event A (et pas A) et Event B (et pas B)) : 
* Si OR = 1, il n’y a pas d’association entre les variables.
* Si OR > 1, l'événement A est plus fréquent en présence de B.
* Si OR < 1, l'événement A est moins fréquent en présence de B.

In [ ]:
from scipy.stats import fisher_exact

oddsratio, p_value = fisher_exact(TableNPM1)

print("\nRésultats du test exact de Fisher:")
print(f"Odds Ratio: {oddsratio}") 
print(f"p-value: {p_value}")




Résultats du test exact de Fisher:
Odds Ratio: 0.016231969419157253
p-value: 0.7118


In [ ]:
import rpy2.robjects as robjects

# Charger la fonction fisher.test de R
fisher_test = robjects.r['fisher.test']

# Convertir la table pour R
r_table = robjects.r.matrix(robjects.FloatVector(np.array(TableNPM1).flatten()), 
                            nrow=TableNPM1.shape[0], 
                            byrow=True)

# Appliquer le test de Fisher généralisé
result = fisher_test(r_table)

# Afficher la p-value
p_value = result.rx2("p.value")[0]
print(f"P-value bilatérale : {p_value}")

# Extraire l’odds ratio et l’intervalle de confiance
if "estimate" in result.names:
    odds_ratio = result.rx2("estimate")[0]  # OR
    conf_int = list(result.rx2("conf.int"))  # IC 95%
    print(f"Odds Ratio : {odds_ratio}")
    print(f"IC 95% : {conf_int}")
else:
    print("Pas d'odds ratio car la table n'est pas 2x2.")

P-value bilatérale : 0.7156819839813257
Pas d'odds ratio car la table n'est pas 2x2.


# DNMT3A

In [ ]:
MutationsCaracteristicsDNMT3A = get_types_mutations(data_DNMT3A_mut, to_csv=True)

In [ ]:
MutationAndPatients_dicoDNMT3A = link_patients_and_mutations(MutationsCaracteristicsDNMT3A, data_DNMT3A_mut)
pprint(OrderedDict(MutationAndPatients_dicoDNMT3A))

OrderedDict([('mut1',
              ['BA2303',
               'BA2924',
               'BA3083',
               'BA3089',
               'BA3146',
               'BA3164',
               'BA3183',
               'BA3192',
               'BA3208',
               'BA3232',
               'BA3235',
               'BA3275',
               'BA3373',
               'BA2035',
               'BA2054',
               'BA2101',
               'BA2110',
               'BA2236',
               'BA2239',
               'BA2244',
               'BA2288',
               'BA2312',
               'BA2355',
               'BA2369',
               'BA2402',
               'BA2405',
               'BA2477',
               'BA2496',
               'BA2512',
               'BA2521',
               'BA2528',
               'BA2593',
               'BA2603',
               'BA2735',
               'BA2772',
               'BA2782',
               'BA2813',
               'BA2820',
               'BA2850',
   

In [ ]:
ind_DNMT3Amut = get_patients_with_gene_mut(data_DNMT3A_mut)
print(ind_DNMT3Amut)
print(len(ind_DNMT3Amut))

['BA2303', 'BA2924', 'BA3083', 'BA3089', 'BA3146', 'BA3164', 'BA3183', 'BA3192', 'BA3208', 'BA3232', 'BA3235', 'BA3275', 'BA3373', 'BA2035', 'BA2054', 'BA2101', 'BA2110', 'BA2236', 'BA2239', 'BA2244', 'BA2288', 'BA2312', 'BA2355', 'BA2369', 'BA2402', 'BA2405', 'BA2477', 'BA2496', 'BA2512', 'BA2521', 'BA2528', 'BA2593', 'BA2593', 'BA2603', 'BA2603', 'BA2735', 'BA2772', 'BA2782', 'BA2813', 'BA2820', 'BA2820', 'BA2850', 'BA2851', 'BA2908', 'BA2914', 'BA2986', 'BA2986', 'BA2989', 'BA2993', 'BA2999', 'BA3006', 'BA3029', 'BA3043', 'BA2971', 'BA3099', 'BA3124', 'BA3195', 'BA3240', 'BA3266', 'BA3270', 'BA3293', 'BA3371', 'BA3421', 'BA3453', 'BA3454', 'BA2065', 'BA2065', 'BA2079', 'BA2139', 'BA2221', 'BA2245', 'BA2249', 'BA2269', 'BA2316', 'BA2452', 'BA2474', 'BA2523', 'BA2523', 'BA2579', 'BA2588', 'BA2662', 'BA2676', 'BA2688', 'BA2742', 'BA2746', 'BA2829', 'BA2901', 'BA3075', 'BA3157', 'BA3176', 'BA2724', 'BA3194', 'BA3202', 'BA3202', 'BA2088', 'BA2088', 'BA3219', 'BA3243', 'BA3255', 'BA3268',

On vérifie que tous les ID sample du fichier de mutations de DNMT3A sont bien tous dans les données de BEAT AML.

In [ ]:
for ind in ind_DNMT3Amut:
    if ind not in ind_beatAML:
        print(ind)
#Renvoie rien : OK

In [ ]:
TableDNMT3A = create_contingency_table(MutationAndPatients_dicoDNMT3A, ind_DNMT3Amut)
TableDNMT3A

/tmp/ipykernel_6077/3046104937.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Table = Table.fillna(0)


,Adverse,Favorable,FavorableOrIntermediate,Intermediate,IntermediateOrAdverse,MissingKaryo,MissingMutations,NonAML,NonInitial
mut1,6,13,3,11,1,1,2,1,20
mut2,2,3,1,1,1,1,1,1,5
mut3,1,1,1,1,1,1,1,1,1
mut4,2,1,1,1,1,1,1,1,1
mut5,8,10,1,2,2,1,1,1,10
mut6,6,7,2,7,1,1,1,2,14
mut7,1,1,1,1,1,1,1,1,1
mut8,1,2,1,1,1,1,1,1,1
mut9,1,3,1,1,1,1,1,1,1
mut10,1,1,1,1,1,1,1,1,2


In [ ]:
# Effectuer le test du chi-carré d'indépendance sur la table de contingence globale
chi2, p, dof, expected = chi2_contingency(TableDNMT3A)

# Afficher les résultats du test
print("\nRésultats du test du chi-carré d'indépendance:")
print(f"Chi2: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:")
# print(expected)

# Interprétation des résultats
alpha = 0.05
if p < alpha:
    print("\nIl existe une association significative entre les types de mutations et la variable ELN2017 (rejeter H0).")
else:
    print("\nIl n'existe pas d'association significative entre les types de mutations et la variable ELN2017 (ne pas rejeter H0).")


Résultats du test du chi-carré d'indépendance:
Chi2: 108.28057061441098
p-value: 1.0
Degrees of freedom: 448

Il n'existe pas d'association significative entre les types de mutations et la variable ELN2017 (ne pas rejeter H0).


In [ ]:
# Initialiser un dictionnaire pour stocker les résultats du test exact de Fisher
results = {}

# Effectuer le test exact de Fisher pour chaque mutation
for mutation in TableDNMT3A.index:
    # Créer une table de contingence pour la mutation actuelle (1 vs all)
    mutation_counts = TableDNMT3A.loc[mutation]
    other_counts = TableDNMT3A.drop(mutation).sum()
    
    mutation_table = pd.DataFrame({
        'Feature': TableDNMT3A.columns,
        'Mutation': mutation_counts,
        'Others': other_counts
    }).set_index('Feature').T

    # Normalisation (évite l'effet du nombre inégal de patients par mutation)
    # mutation_table = mutation_table.div(mutation_table.sum(axis=1), axis=0)
    # print(mutation_table)
    
    # Test exact de Fisher pour chaque catégorie de la feature
    fisher_results = {}
    for category in mutation_table.columns:
        # Construire la table 2x2 pour le test de Fisher
        table_2x2 = np.array([
            [mutation_table.loc["Mutation", category], mutation_table.loc["Mutation"].sum() - mutation_table.loc["Mutation", category]],
            [mutation_table.loc["Others", category], mutation_table.loc["Others"].sum() - mutation_table.loc["Others", category]]
        ])

        # Vérifier que la table contient des valeurs non nulles
        if np.all(table_2x2 > 0):
            odds_ratio, p_fisher = fisher_exact(table_2x2, alternative="two-sided")
            fisher_results[category] = {"Odds Ratio": odds_ratio, "p-value (Fisher)": p_fisher}
        else:
            fisher_results[category] = {"Odds Ratio": None, "p-value (Fisher)": None}

    # Ajouter les résultats de Fisher au dictionnaire principal
    results[mutation] = fisher_results

# Affichage des résultats
# for mutation, fisher_result in results.items():
#     print(f"Mutation: {mutation}")
#     for category, result in fisher_result.items():
#         print(f"  {category}: OR={result['Odds Ratio']}, p-value={result['p-value (Fisher)']}")
#     print("\n")

# Interprétation des résultats (seuil alpha = 0.05)
alpha = 0.05
for mutation, fisher_result in results.items():
    for category, result in fisher_result.items():
        if result['p-value (Fisher)'] is not None and result['p-value (Fisher)'] < alpha:
            print(f"La catégorie {category} est significativement associée à la mutation {mutation} selon le test exact de Fisher.")

  → La catégorie IntermediateOrAdverse est significativement associée à la mutation mut1 selon le test exact de Fisher.
  → La catégorie NonAML est significativement associée à la mutation mut1 selon le test exact de Fisher.
  → La catégorie NonInitial est significativement associée à la mutation mut1 selon le test exact de Fisher.
  → La catégorie Favorable est significativement associée à la mutation mut5 selon le test exact de Fisher.
  → La catégorie NonInitial est significativement associée à la mutation mut6 selon le test exact de Fisher.


# NPM1 et DNMT3A ?

In [ ]:
patient_bothMutated = []

for ind in ind_NPM1mut:
    if ind in ind_DNMT3Amut:
        patient_bothMutated.append(ind)

print(patient_bothMutated)
print(len(patient_bothMutated))

['BA2303', 'BA2924', 'BA3083', 'BA3089', 'BA3099', 'BA3157', 'BA3164', 'BA3176', 'BA3183', 'BA3192', 'BA3232', 'BA3235', 'BA3258', 'BA3266', 'BA3293', 'BA3297', 'BA3328', 'BA3385', 'BA3421', 'BA3446', 'BA3453', 'BA3454', 'BA2003', 'BA2032', 'BA2054', 'BA2065', 'BA2106', 'BA2110', 'BA2123', 'BA2139', 'BA2171', 'BA2200', 'BA2236', 'BA2239', 'BA2244', 'BA2249', 'BA2376', 'BA2418', 'BA2452', 'BA2474', 'BA2477', 'BA2512', 'BA2523', 'BA2552', 'BA2579', 'BA2588', 'BA2662', 'BA2704', 'BA2746', 'BA2772', 'BA2782', 'BA2789', 'BA2806', 'BA2813', 'BA2820', 'BA2829', 'BA2893', 'BA2910', 'BA2914', 'BA2939', 'BA3006', 'BA3043', 'BA3053', 'BA3075', 'BA3146', 'BA2068', 'BA3219', 'BA3270', 'BA3386', 'BA2603', 'BA2724', 'BA2735', 'BA2901', 'BA3029', 'BA3255', 'BA3335', 'BA2521']
77


In [ ]:
# Plus de recherches sur le test stastistique exact de Fisher => quel n de significativité ?
# Utiliser ce n comme seuil pour exclure les types de mutations qui n'ont pas assez de patients qui l'ont

# => Refaire avec n=3 le test du chi2 (et Fisher)